## Matrices

Matrices are very common in transportation.  Two commont types of matrix data are: 

trip tables - matrices showing the number of trips between each zone pair

skims - matrices showing the travel time and cost between each zone pair


In [1]:
# you can represent matrices in python as two dimensional lists
m = [[1,2,3],[4,5,6],[7,8,9]]
m

[[1, 2, 3], [4, 5, 6], [7, 8, 9]]

In [2]:
# change the values by referring to the appropriate indices
m[0][0] = -1
m[1][2] = 999
m

[[-1, 2, 3], [4, 5, 999], [7, 8, 9]]

### Numpy matrices

This is fine, but can be slow if you're working with big matrices.  For those, it is nice to use numpy, which operates way faster.  Their quickstart tutorial offers a good intro: 

https://docs.scipy.org/doc/numpy-dev/user/quickstart.html

In [3]:
import numpy as np
m = np.array([[1,2,3],[4,5,6],[7,8,9]])
m

array([[1, 2, 3],
       [4, 5, 6],
       [7, 8, 9]])

In [ ]:
# change the values by referring to the appropriate indices
m[0][0] = -1
m[1][2] = 999
m

In [4]:
# Since an array is an object, it comes with some extra data 
m.shape

(3, 3)

In [5]:
# and some methods
m.max()

9

### OMX for matrix i/o

For saving groups of matrices to disk, OMX is a useful package.  Intro is here: 

https://github.com/osPlanning/omx/wiki

Python API is here: 

https://github.com/osPlanning/omx-python

Note that we sometimes want row and column indices that are non-sequential integers--the label of the TAZ rather than just a number 1 through N.  I've put an example matrix in the data folder for this lesson.  If you want to look at the contents, you can install the viewer, available on the wiki.  


### Homework

The example is a skim matrix showing different components of travel time between each TAZ pair.  The IVT matrix is for in-vehicle time (as opposed to walking and waiting time which are out-of-vehicle).  Notice that the values on the diagonal are all zeros.  This is the sort of thing that can cause divide by zero errors later on.  Your job is to read the matrix in, replace the zeros on the diagonals with ones, and save the matrix again.  

To do this, you will need to install the OMX package and review their quick start sample code to see what to do.  This is good practice in figuring stuff out from the types of resources that may be avaiable, and you should have the skills at this point in the course to do so.


In [1]:
from __future__ import print_function
import openmatrix as omx
import numpy as np

In [ ]:
example = omx.open_file('data\example.omx')

In [ ]:
from __future__ import print_function
import openmatrix as omx
import numpy as np

# Create some data
ones = np.ones((100,100))
twos = 2.0*ones


# Create an OMX file (will overwrite existing file!)
print('Creating myfile.omx')
myfile = omx.open_file('myfile.omx','w')   # use 'a' to append/edit an existing file


# Write to the file.
myfile['m1'] = ones
myfile['m2'] = twos
myfile['m3'] = ones + twos           # numpy array math is fast
myfile.close()


# Open an OMX file for reading only
print('Reading myfile.omx')
myfile = omx.open_file('myfile.omx')

print ('Shape:', myfile.shape())                 # (100,100)
print ('Number of tables:', len(myfile))         # 3
print ('Table names:', myfile.list_matrices())   # ['m1','m2',',m3']


# Work with data. Pass a string to select matrix by name:
# -------------------------------------------------------
m1 = myfile['m1']
m2 = myfile['m2']
m3 = myfile['m3']

# halves = m1 * 0.5  # CRASH!  Don't modify an OMX object directly.
#                    # Create a new numpy array, and then edit it.
halves = np.array(m1) * 0.5

first_row = m2[0]
first_row[:] = 0.5 * first_row[:]

my_very_special_zone_value = m2[10][25]


# FANCY: Use attributes to find matrices
# --------------------------------------
myfile.close()                            # was opened read-only, so let's reopen.
myfile = omx.open_file('myfile.omx','a')  # append mode: read/write existing file

myfile['m1'].attrs.timeperiod = 'am'
myfile['m1'].attrs.mode = 'hwy'

myfile['m2'].attrs.timeperiod = 'md'

myfile['m3'].attrs.timeperiod = 'am'
myfile['m3'].attrs.mode = 'trn'

print('attributes:', myfile.list_all_attributes())       # ['mode','timeperiod']

# Use a DICT to select matrices via attributes:

all_am_trips = myfile[ {'timeperiod':'am'} ]                    # [m1,m3]
all_hwy_trips = myfile[ {'mode':'hwy'} ]                        # [m1]
all_am_trn_trips = myfile[ {'mode':'trn','timeperiod':'am'} ]   # [m3]

print('sum of some tables:', np.sum(all_am_trips))


# SUPER FANCY: Create a mapping to use TAZ numbers instead of matrix offsets
# --------------------------------------------------------------------------
# (any mapping would work, such as a mapping with large gaps between zone
#  numbers. For this simple case we'll just assume TAZ numbers are 1-100.)

taz_equivs = np.arange(1,101)                  # 1-100 inclusive

myfile.create_mapping('taz', taz_equivs)
print('mappings:', myfile.list_mappings())                 # ['taz']

tazs = myfile.mapping('taz')          # Returns a dict:  {1:0, 2:1, 3:2, ..., 100:99}

m3 = myfile['m3']

print('cell value:', m3[tazs[100]][tazs[100]])      # 3.0  (taz (100,100) is cell [99][99])

myfile.close()